# Day 13: Shuttle Search

[*Advent of Code 2020 day 13*](https://adventofcode.com/2020/day/13) and [*solution megathread*](https://redd.it/kc4njx)

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/UncleCJ/advent-of-code/blob/cj/2020/13/code.ipynb) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/UncleCJ/advent-of-code/cj?filepath=2020%2F13%2Fcode.ipynb)

In [1]:
from IPython.display import HTML
import sys
sys.path.append('../../')
import common

downloaded = common.refresh()
%store downloaded >downloaded

Writing 'downloaded' (dict) to file 'downloaded'.


## Part One

In [2]:
HTML(downloaded['part1'])

## Boilerplate

Let's try using [pycodestyle_magic](https://github.com/mattijn/pycodestyle_magic) with pycodestyle (flake8 stopped working for me in VS Code Jupyter). Now how does type checking work?

In [3]:
%load_ext pycodestyle_magic

In [4]:
%pycodestyle_on

In [5]:
testdata = """939
7,13,x,x,59,x,31,19""".splitlines()

inputdata = downloaded['input'].splitlines()

In [6]:
from itertools import count
from functools import reduce
import numpy as np


def process_data(data):
    return (int(data[0]),
            [(int(id), o) for o, id in
                enumerate(data[1].split(','))
                if id != 'x'])


def waiting_times(earliest, ids):
    return [(0, id) if (earliest % id == 0)
            else (id - (earliest % id), id)
            for id, o in ids]

In [7]:
earliest, ids = process_data(testdata)
print(earliest, ids)
times = waiting_times(earliest, ids)
print(f'times: {times}')
shortest = min(times)
print(f'shortest waiting time * that bus Id: '
      f'{shortest[0]}*{shortest[1]} = {shortest[0]*shortest[1]}')

939 [(7, 0), (13, 1), (59, 4), (31, 6), (19, 7)]
times: [(6, 7), (10, 13), (5, 59), (22, 31), (11, 19)]
shortest waiting time * that bus Id: 5*59 = 295


In [8]:
earliest, ids = process_data(inputdata)
print(earliest, ids)
times = waiting_times(earliest, ids)
print(f'times: {times}')
shortest = min(times)
print(f'shortest waiting time * that bus Id: '
      f'{shortest[0]}*{shortest[1]} = {shortest[0]*shortest[1]}')

1000677 [(29, 0), (41, 19), (661, 29), (13, 42), (17, 43), (23, 52), (521, 60), (37, 66), (19, 79)]
times: [(26, 29), (10, 41), (77, 661), (11, 13), (11, 17), (7, 23), (164, 521), (25, 37), (15, 19)]
shortest waiting time * that bus Id: 7*23 = 161


In [9]:
HTML(downloaded['part1_footer'])

## Part Two

In [10]:
HTML(downloaded['part2'])

In [11]:
def print_schedule(earliest, ids):
    wtimes_ids = waiting_times(earliest, ids)
    wtimes = [wtime for wtime, id in wtimes_ids]
    ids = [id for id, o in ids]
    print(f'time    {"  ".join(["bus " + str(id) for id in ids])}')
    for t in range(0, max(wtimes) + 1):
        lines = [" D " if (t + earliest) % id == 0
                 else " . "
                 for id in ids]
        print(f'{t + earliest}    {"     ".join(lines)}')

Here first I went down the wrong rabbit hole, Euclid's lemma and the Extended Greatest Common Divisor algorithm. See below for what actually worked out.

For $B_a$ and $B_b$ as two bus ids in this context (also their period), where $B_a$ is assumed to have $o_a$ offset (or "phase") and $B_b$ offset $o_b$, let $d_{a,b} = gcd(B_a, B_b)$

By [Euclid's lemma](https://en.wikipedia.org/wiki/Euclid%27s_lemma), there exist integers $N_a, N_b$ such that $N_a ⋅ B_a - N_b ⋅ B_b = c$

We are looking for $m, n$ so that $m⋅B_a − o_a = n⋅B_b − o_b => m⋅B_a − n⋅B_b = o_a − o_b$ $(1)$. Using Euclid's extended algorithm, we can find $s, t$ so that $s⋅B_a + t⋅B_b = gcd(B_a, B_b)$ $(2)$. Assuming $mod(o_a - o_b, g_{a,b}) = 0$ (there exists a solution), let $z = \frac{o_a - o_b}{g_{a,b}}$, multiply $(2)$ with it to get $(1)$ as $m = z⋅s$ and $n = −z⋅t$

In [12]:
def extended_gcd(a, b):
    """Extended Greatest Common Divisor Algorithm

    Returns:
        gcd: The greatest common divisor of a and b.
        s, t: Coefficients such that s*a + t*b = gcd

    Reference:
        https://en.wikipedia.org/wiki/Extended_Euclidean_algorithm#Pseudocode
    """
    old_r, r = a, b
    old_s, s = 1, 0
    old_t, t = 0, 1
    while r:
        quotient, remainder = divmod(old_r, r)
        old_r, r = r, remainder
        old_s, s = s, old_s - quotient * s
        old_t, t = t, old_t - quotient * t

    return old_r, old_s, old_t


def combine_phased_schedules(a_period, a_phase, b_period, b_phase):
    gcd, s, t = extended_gcd(a_period, b_period)
    phase_difference = a_phase - b_phase
    pd_mult, pd_remainder = divmod(phase_difference, gcd)
    if pd_remainder:
        raise ValueError("Schedules never synchronize")
    combined_period = a_period // gcd * b_period
    combined_phase = (a_phase - s * pd_mult * a_period) % combined_period
    return combined_period, combined_phase


def schedule_alignment(id_a, id_b):
    a_period, a_offset = id_a
    b_period, b_offset = id_b
    """Where the arrows first align, where green starts shifted by advantage"""
    period, phase = combine_phased_schedules(a_period, a_offset % a_period,
                                             b_period, b_offset % b_period)
    print(period, -phase % period)
    return (period, -phase % period)

Even I can't decipher this frenetic debugging:

In [13]:
# print(schedule_alignment(ids[0], ids[1]))
# print(schedule_alignment(ids[2], ids[3]))
# print(schedule_alignment((91, 77), (1829, 645)))
# print(schedule_alignment((166439, 96292), ids[4]))

# print(reduce(lambda a, b: schedule_alignment(a, b), ids))
# print_schedule(1068781, ids)

# print(schedule_alignment(166439, 98357, 19, 7))
# idx = [0, 1, 2, 3, 4]
# print_schedule(3162341+1068781, np.array(ids)[idx])
# 2093560

earliest, ids = process_data(inputdata)
print(ids)

print(
    schedule_alignment(
        schedule_alignment(
            schedule_alignment(
                schedule_alignment(
                    (29, 0), (41, 19)),
                schedule_alignment(
                    (661, 29), (13, 42))),
            schedule_alignment(
                schedule_alignment(
                    (17, 43), (23, 52)),
                schedule_alignment(
                    (521, 60), (37, 66)))),
        (19, 79)))

[(29, 0), (41, 19), (661, 29), (13, 42), (17, 43), (23, 52), (521, 60), (37, 66), (19, 79)]
1189 145
8593 8564
10217077 2491999
391 178
19277 18175
7537307 5668540
77009245991639 59872140247540
1463175673841141 864238811652128
(1463175673841141, 864238811652128)


Admittedly, in the [solution megathread](https://redd.it/kc4njx), and there's no denying the impact of the appropriate mathematical hint. The above attempts were thrown aside and the solution worked fairly quickly.

In [14]:
def chinese_remainder(n, a):
    sum = 0
    prod = reduce(lambda a, b: a*b, n)
    for n_i, a_i in zip(n, a):
        p = prod // n_i
        sum += a_i * mul_inv(p, n_i) * p
    return sum % prod


def mul_inv(a, b):
    b0 = b
    x0, x1 = 0, 1
    if b == 1:
        return 1
    while a > 1:
        q = a // b
        a, b = b, a % b
        x0, x1 = x1 - q * x0, x0
    if x1 < 0:
        x1 += b0
    return x1


def offset_errors(time, ids):
    return [(0, id) if (time % id) == o
            else (((time % id) + o) % id, id)
            for id, o in ids]

Here things are starting to look more sensible:

In [15]:
print(offset_errors(864238811652128, ids))

print(chinese_remainder([id for id, o in ids], [o for id, o in ids]))

# print_schedule(864238811652128, ids)
# for t in count(0, max(ids)[0]):
#     #print_schedule(t, ids)
#     wtimes_ids = waiting_times(t, ids)
#     errors = [o - t for t, o in zip([wtime for wtime, id in wtimes_ids],
#                                     [o for id, o in ids])]
#     if sum([abs(e) for e in errors]) == 0:
#         print(errors)
#         print_schedule(t, ids)
#         break

[(0, 29), (0, 41), (0, 661), (6, 13), (1, 17), (12, 23), (0, 521), (21, 37), (0, 19)]
1249285041610323


In [16]:
earliest, ids_os = process_data(testdata)
# print(ids_os)
ids = [id for id, _ in ids_os]
os_neg = [-o for _, o in ids_os]
print(f'chinese remainder: {chinese_remainder(ids, os_neg)}')
# print(1068781 % ids_os[3][0])
print(offset_errors(1068781, ids_os))

chinese remainder: 1068781
[(0, 7), (0, 13), (0, 59), (0, 31), (0, 19)]


In [17]:
earliest, ids_os = process_data(inputdata)
ids = [id for id, _ in ids_os]
os_neg = [-o for _, o in ids_os]
print(f'chinese remainder: {chinese_remainder(ids, os_neg)}')
print(offset_errors(213890632230818, ids_os))

chinese remainder: 213890632230818
[(0, 29), (0, 41), (0, 661), (0, 13), (0, 17), (0, 23), (0, 521), (0, 37), (0, 19)]


In [18]:
HTML(downloaded['part2_footer'])